<h1> Best Grades </h1>

I first wrote this code, which takes the data from a git branch calles "master"

In [5]:
import requests

def get_course_statistics(course_number):
    # Construct the GitHub API URL for the course directory
    api_url = f"https://api.github.com/repos/michael-maltsev/technion-histograms/contents/{course_number}"
    
    try:
        # Fetch list of contents in the course directory
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for bad response status
        contents = response.json()

        statistics = {}

        # Loop through each content in the directory
        for item in contents:
            if item['type'] == 'dir':
                folder_name = item['name']
                finals_json_url = f"https://raw.githubusercontent.com/michael-maltsev/technion-histograms/master/{course_number}/{folder_name}/Finals.json"

                # Check if Finals.json exists in the folder
                finals_response = requests.head(finals_json_url)
                if finals_response.status_code == 200:
                    finals_data = requests.get(finals_json_url).json()
                    statistics[folder_name] = {
                        'average': finals_data.get('average'),
                        'median': finals_data.get('median')
                    }

        return statistics
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

# Example usage:
course_number = input("Enter the course number: ")
course_statistics = get_course_statistics(course_number)
if course_statistics:
    for folder, stats in course_statistics.items():
        print(f"Folder: {folder}, Average: {stats['average']}, Median: {stats['median']}")


Enter the course number: 336022
Folder: 201501, Average: 81.552, Median: 84
Folder: 201601, Average: 85.118, Median: 88.5
Folder: 201701, Average: 77.279, Median: 78
Folder: 201801, Average: 79.404, Median: 82.5
Folder: 201901, Average: 77.947, Median: 78
Folder: 202001, Average: 84.282, Median: 87
Folder: 202101, Average: 84.253, Median: 86
Folder: 202201, Average: 84.7, Median: 87


As it turned out to be outdated, I changed the branch to "main" (https://github.com/michael-maltsev/technion-histograms/tree/main)
- and this is to a single course

In [2]:
import requests

def get_course_statistics(course_number):
    # Construct the GitHub API URL for the course directory
    api_url = f"https://api.github.com/repos/michael-maltsev/technion-histograms/contents/{course_number}"
    
    try:
        # Fetch list of contents in the course directory
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for bad response status
        contents = response.json()

        statistics = {}

        # Loop through each content in the directory
        for item in contents:
            if item['type'] == 'dir':
                folder_name = item['name']
                finals_json_url = f"https://raw.githubusercontent.com/michael-maltsev/technion-histograms/main/{course_number}/{folder_name}/Finals.json"

                # Check if Finals.json exists in the folder
                finals_response = requests.head(finals_json_url)
                if finals_response.status_code == 200:
                    finals_data = requests.get(finals_json_url).json()
                    statistics[folder_name] = {
                        'average': finals_data.get('average'),
                        'median': finals_data.get('median')
                    }

        return statistics
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

# Example usage:
course_number = input("Enter the course number: ")
course_statistics = get_course_statistics(course_number)
if course_statistics:
    for folder, stats in course_statistics.items():
        print(f"Folder: {folder}, Average: {stats['average']}, Median: {stats['median']}")


Enter the course number: 336002
Error fetching data: 404 Client Error: Not Found for url: https://api.github.com/repos/michael-maltsev/technion-histograms/contents/336002


<h2>Signals & Imaging courses: must complete at least 5 courses</h2>

Core (must complete at least 2)

In [3]:
336020
336023
336027
336208
336326
336502
336522
336533

336533

Not Core

In [6]:
"""336028
336325
336404
336504
336521
336536
336538
336540
336544
336545
336546
336547 
336549
336550
034033
044101
044198
044202
046195
046197
046200
046201
046211
046332
046745
046746
236523
236781"""

'336028\n336325\n336404\n336504\n336521\n336536\n336538\n336540\n336544\n336545\n336546\n336547 \n336549\n336550\n034033\n044101\n044198\n044202\n046195\n046197\n046200\n046201\n046211\n046332\n046745\n046746\n236523\n236781'

In [7]:
otot = ["336020","336023","336027","336208","336326","336502","336522","336533","336325","336504","336521","336536","336538",
"336540","336544","336545","336546","336547","336549","336550","034033","044101","044198","044202","046195","046197","046200",
"046201","046211","046332","046745","046746","236523","236781"]

In [8]:
Mechanica = "336326 336506 336517 336522 336539 336541 336021 336520 336521 336540 336549 336550 034033 035001 035199 036049  036076  084225"

In [9]:
tissue = "336021 ננ - לויבב םיקיקלח ' יגולואירו הקינכמ 2 1 - 2.5 336402 ןכת ויב - םירמוח 2 1 - 2.5 336517 ויב - אתה לש הסדנה 2 1 - 2.5 336528 תופורת לש רקובמ רורחש 2 1 - 2.5 336529 םייגולויב םיפילחתו תומקר תסדנה 2 1 - 2.5 336326 םירטמרפ ךורעשו םינותנ חותינ 2 1 - 2.5 336404 םיינורטקלאויב םיקשממ 2 1 - 2.5 336405 כטויבבו היגולויבב םייסדנה תודוסי 'נ 2 1 - 2.5 336520 המקר יפילחתו םיידפוטרוא םילתש 2 1 - 2.5 336521 וידרקה 'עמ לש םייסדנה תונורקע 3 1 - 3.5 336538 ויב תונורקע - תולוקלומ תשיחל םייסדנה 2 1 - 2.5 336544 םייטנג םילגעמ 2 1 - 2.5 336548 ןכתל הדבעמ םייטנג םילגעמ - - 4 2.0 336549 ףוציר תוקינכט DNA 2 2 - 3.0 336550 תיבושיח הקיזיפויב 2 1 - 2.5 054413 היגולונכטויבב םהימושיו םירמילופ 2 1 - 2.5 134020 תיללכ הקיטנג 3 1 - 3.5 134069 תוחתפתהה לש היגולויב 2 1 - 2.5 134082 תירלוקלומ היגולויב 2 1 - 2.5 134119 יטנגה יוטיבה תרקב 2 1 - 2.5 134121 היגולוריוו היגולויבורקימ 3 - - 3.0 276413"

In [10]:
hova_2024 = "044102 למשח תודבעמב תוחיטב 4 - - 0.0 104065 הרבגלא 1מ2 4 2 - 5.0 104042 ודח 1מ2 4 2 - 5.0 114071 הקיזיפ 1 'מ 3 1 - 3.5 125001  הימיכ תיללכ 2 2 - 3.0 134058 104013 אודח 2ת 4 3 - 5.5 104038 הרבגלא 2מ 2 1 - 2.5 104136 רדמ  3 2 - 4.0 114052 הקיזיפ 2 3 1 - 3.5 124801 תינגרוא הימיכ 1 ב 2 1 - 2.5 234128 044105 םיילמשחה םילגעמה תרות 3 2 - 4.0 104214 הירופ ירוט 2 1 - 2.5 104215 תובכורמ תויצקנופ 2 1 - 2.5 104220 דמ 2 1 - 2.5 124503 תילקיזיפ הימיכ 1ב 2 1 - 2.5 134019 היגולומיזנאו הימיכויבל אובמ 2 1 - 2.5 334274 044131 תוכרעמו תותוא 4 2 - 5.0 104034 ח תורבתסהל אובמ 3 3 - 3.5 334221 לש תודוסי םירמוח םייאופר 2 1 - 2.5 334222 םיקצומ תקינכמ ויב הסדנהל - תיאופר 3 2 - 4.0 336537 134113 םיילובטמ םילולסמ 3 1 - 3.5 276011 םיסדנהמל ףוגה תוכרעמ לש היגולויזיפ 3 1 - 3.5 334009 םייגולויב םימרוז תקינכמ 3 2 - 4.0 334011 ויב ןכת תודוסי - ילמשח 3 2 - 4.0 335010 יסיסב ינכמויב ןכת 2 2 - 3.0 336022 334023 הקיטסיטטסל אובמ 3 1 - 3.5 335001 334014 ויב הסדנהב טקיורפ - תיאופר 1 - - 9 4.0 335002 335015 ויב הסדנהב טקיורפ - תיאופר 2 - - 9 3.0 335003"

In [32]:
hova_2021 = "104016 104018 114071 125001 134058 104013 104135 114052 124801 234128 044105 104221 104223 134019 334274 044131 104034 336537 334221 335009 334222 134113 276011 334011 336022 336533 334023 335001 335010 334014 335002 335015 335003"

In [11]:
import re

def find_6_digit_sequences(input_string):
    # Define the regex pattern to match 6-digit sequences
    pattern = r'\b\d{6}\b'
    
    # Find all matches using findall() method
    sequences = re.findall(pattern, input_string)
    
    return sequences

In [12]:
# Example usage:
input_string = Mechanica
mech = find_6_digit_sequences(input_string)

['336326', '336506', '336517', '336522', '336539', '336541', '336021', '336520', '336521', '336540', '336549', '336550', '034033', '035001', '035199', '036049', '036076', '084225']


In [13]:
rekamot = find_6_digit_sequences(tissue)

This code goes through the list of courses and extract all the stats with a progress bar

In [40]:
import csv
from tqdm import tqdm
from time import sleep

def get_finals_statistics(course_numbers):
    all_statistics = {}
    for course_number in tqdm(course_numbers):
        # Construct the GitHub API URL for the course directory
        api_url = f"https://api.github.com/repos/michael-maltsev/technion-histograms/contents/{course_number}"
        
        try:
            # Fetch list of contents in the course directory
            response = requests.get(api_url)
            response.raise_for_status()  # Raise an exception for bad response status
            contents = response.json()

            # Loop through each content in the directory
            for item in contents:
                if item['type'] == 'dir':
                    folder_name = item['name']
                    finals_json_url = f"https://raw.githubusercontent.com/michael-maltsev/technion-histograms/main/{course_number}/{folder_name}/Finals.json"

                    # Check if Finals.json exists in the folder
                    finals_response = requests.head(finals_json_url)
                    if finals_response.status_code == 200:
                        finals_data = requests.get(finals_json_url).json()
                        if course_number not in all_statistics:
                            all_statistics[course_number] = {'medians': [], 'folders': {}}
                        all_statistics[course_number]['medians'].append(finals_data.get('median'))
                        all_statistics[course_number]['folders'][folder_name] = {
                            'average': finals_data.get('average'),
                            'median': finals_data.get('median')
                        }
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for course {course_number}: {e}")
        sleep(0.02)
    return all_statistics

In [41]:
def write_to_csv(statistics):
    filename = input("Enter the filename to save the CSV: ")
    filename = filename + ".csv"
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Course Number', 'Folder', 'Average', 'Median'])
        for course_number, data in statistics.items():
            for folder, stats in data['folders'].items():
                writer.writerow([course_number, folder, stats['average'], stats['median']])
        writer.writerow(['Course Number', 'Average of Medians'])
        for course_number, data in statistics.items():
            medians = [median for median in data['medians'] if isinstance(median, (int, float))]
            if medians:
                average_median = sum(medians) / len(medians)
            else:
                average_median = "N/A"
            writer.writerow([course_number, average_median])
    print(f"Data has been written to {filename}")

In [ ]:
# Example usage:
course_numbers = otot  # Replace with your list of course numbers
all_statistics = get_finals_statistics(course_numbers)
write_to_csv(all_statistics)

In [ ]:
mech_stats = get_finals_statistics(mech)
write_to_csv(mech_stats)

In [ ]:
reka_stats = get_finals_statistics(rekamot)
write_to_csv(reka_stats)

In [16]:
main_stats = get_finals_statistics(main)
write_to_csv(main_stats)

  6%|██▍                                        | 2/36 [00:14<03:32,  6.24s/it]

Error fetching data for course 104065: 404 Client Error: Not Found for url: https://api.github.com/repos/michael-maltsev/technion-histograms/contents/104065


100%|██████████████████████████████████████████| 36/36 [05:33<00:00,  9.26s/it]


Enter the filename to save the CSV: main_stats
Data has been written to main_stats


In [37]:
main2021 = get_finals_statistics(find_6_digit_sequences(hova_2021))
wrtie_to_csv(main2021)

 94%|███████████████████████████████████████▍  | 31/33 [05:59<00:08,  4.45s/it]

Error fetching data for course 335002: 403 Client Error: rate limit exceeded for url: https://api.github.com/repos/michael-maltsev/technion-histograms/contents/335002


 97%|████████████████████████████████████████▋ | 32/33 [06:00<00:03,  3.20s/it]

Error fetching data for course 335015: 403 Client Error: rate limit exceeded for url: https://api.github.com/repos/michael-maltsev/technion-histograms/contents/335015


100%|██████████████████████████████████████████| 33/33 [06:00<00:00, 10.93s/it]

Error fetching data for course 335003: 403 Client Error: rate limit exceeded for url: https://api.github.com/repos/michael-maltsev/technion-histograms/contents/335003


NameError: name 'wrtie_to_csv' is not defined

In [39]:
write_to_csv(main2021)

Enter the filename to save the CSV: main2021
Data has been written to main2021.csv


Data has been written to temp_data.csv
